In [1]:
import numpy as np
from rubin_sim.satellite_constellations import Constellation, starlink_tles_v1, starlink_tles_v2
import pandas as pd
from rubin_sim.utils import point_to_line_distance

In [2]:
# need to add a new method on the constallation to do the checking the way we want
class SConstellation(Constellation):
    def check_positions(self, positions_ra, positions_dec, mjds, visit_times, mask_width=1./60.):
        """Check if RA,dec,mjd spot is hit by a satellite streak
        
        Parameters
        ----------
        positions_ra : np.array
            The RA of each solar system detection (degrees).
        positions_dec : np.array
            The Dec of each solar system detection (degrees).
        mjds : np.array
            The MJD of each detection
        visit_times : np.array
            The total visit times for each detection. Typically exposure time plus 
            any additional read time or shutter motion time. (seconds)
        mask_width : float
            The width of the expected streak (degrees)
        
        Returns
        -------
        index values for positions that were hit by a streak
        """
        # convert all input to radians
        positions_ra = np.radians(positions_ra)
        positions_dec = np.radians(positions_dec)
        mask_width = np.radians(mask_width)
        
        visit_times = visit_times / 3600.0 / 24.0  # convert seconds to days
        
        input_id_indx_oned = np.arange(positions_ra.size, dtype=int)
        
        sat_ra_1, sat_dec_1, sat_alt_1, sat_illum_1 = self.paths_array(mjds)
        mjd_end = mjds + visit_times
        sat_ra_2, sat_dec_2, sat_alt_2, sat_illum_2 = self.paths_array(mjd_end)
    
        
        # broadcast the object positions to be the same shape as the satellite arrays.
        pointing_ras_rad = np.broadcast_to(positions_ra, sat_ra_1.shape)
        pointing_decs_rad = np.broadcast_to(positions_dec, sat_ra_1.shape)
        input_id_indx = np.broadcast_to(input_id_indx_oned, sat_ra_1.shape)
        
        above_illum_indx = np.where(
            ((sat_alt_1 > self.alt_limit_rad) | (sat_alt_2 > self.alt_limit_rad))
            & ((sat_illum_1 == True) | (sat_illum_2 == True))
        )

        # XXXX--this is assuming satellites travel on straight lines on the sphere
        # I'm not sure that's a super great assumption, but maybe works out
        # statistically where we get some hits right and some wrong
        
        # point_to_line_distance can take arrays, but they all need to be the same shape,
        # thus why we broadcast pointing ra and dec above.
        # This might be better done with a KD tree. Especially if cranking up to 
        # very large satellite constellations.
        distances = point_to_line_distance(
            sat_ra_1[above_illum_indx],
            sat_dec_1[above_illum_indx],
            sat_ra_2[above_illum_indx],
            sat_dec_2[above_illum_indx],
            pointing_ras_rad[above_illum_indx],
            pointing_decs_rad[above_illum_indx],
        )

        close = np.where(distances < mask_width)[0]
        
        # Could set a "close pass" value, maybe one degree, use that instead of the
        # mask width above, then for each indx in close, do a higher time-resolution
        # calculation of the satllite path and check if it gets within mask_width.
        # there is an example in the original check_pointings method on the base class
        
        # Note, could have repeat values in result as an object can be hit by more than one streak
        return input_id_indx[above_illum_indx][close]

In [3]:
ss_observations_file = 'baseline_v3.0_10yrs__vatiras_granvik_10k_obs.txt'
ss_objects = pd.read_csv(ss_observations_file, delim_whitespace=True, comment="#")

In [4]:
ss_objects

,obj_id,time,ra,dec,dradt,ddecdt,phase,solarelon,helio_dist,geo_dist,...,night,observationStartMJD,rotSkyPos,seeingFwhmEff,seeingFwhmGeom,solarElong,visitExposureTime,dmag_color,dmag_trail,dmag_detect
0,11,62967.414922,338.402378,-0.093060,0.499458,0.651728,90.777979,41.582800,0.665119,0.740436,...,2749,62967.414922,49.883936,1.269010,1.095126,42.944912,30.0,-0.399899,0.150709,0.170063
1,11,62968.416111,338.928469,0.562650,0.533739,0.658322,90.423639,41.886436,0.669232,0.741200,...,2750,62968.416111,79.752346,1.251011,1.080331,41.662135,30.0,-0.399899,0.159705,0.183700
2,22,63674.401541,329.366527,-33.677273,0.801230,0.375649,82.593321,42.227152,0.674560,0.823929,...,3456,63674.401541,181.241227,1.340649,1.154013,42.262080,30.0,-0.399899,0.155128,0.176724
3,22,63675.408781,330.338429,-33.279460,0.802071,0.397036,81.883445,42.236281,0.676011,0.832516,...,3457,63675.408781,174.853887,1.205235,1.042703,42.154766,30.0,-0.399899,0.178728,0.213525
4,26,62260.419611,0.143246,3.998665,0.775070,0.507439,103.043368,42.223382,0.695625,0.589708,...,2042,62260.419611,147.127361,1.728888,1.473146,43.476731,30.0,-0.399899,0.119291,0.124996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6098,9976,62154.045303,344.775968,-38.403037,0.772214,0.023012,90.254853,42.408950,0.663595,0.723482,...,1936,62154.045303,246.425257,1.635481,1.396366,41.978399,15.0,-0.457036,0.034142,0.026573
6099,9989,62924.401263,298.192098,-11.913906,1.004475,0.338591,71.885575,41.064252,0.684474,0.959435,...,2706,62924.401263,180.190662,1.550441,1.326463,41.958570,30.0,-0.264722,0.163490,0.189529
6100,9990,60314.358347,235.417535,-34.766982,1.147239,-0.137481,93.028481,44.958637,0.695818,0.659032,...,96,60314.358347,84.968832,1.643236,1.402740,46.486202,15.0,-0.264722,0.064519,0.057213
6101,9990,60314.361440,235.421854,-34.767407,1.146943,-0.137357,93.028284,44.958045,0.695811,0.659042,...,96,60314.361440,84.493836,1.425783,1.223993,46.489140,15.0,-0.264722,0.079210,0.073917


In [5]:
tles = starlink_tles_v2()
print('N satellites = ', np.size(tles))
constellation = SConstellation(tles)

N satellites =  29988


[#################################] 100% de421.bsp


In [6]:
visit_times = ss_objects['visitExposureTime']+2


In [7]:
# this step is the grind. Generates a few arrays that are n_detections X n_satellites, so
# potential to gobble up a few GB of memory. Looks like I got up to 10-15 GB on a 6.8k x 30k run.
indx = constellation.check_positions(ss_objects['ra'].values, ss_objects['dec'].values,
                                     ss_objects['observationStartMJD'].values, visit_times.values)
# Note that if you want to check something like a 300k constellation, one can break it up into 
# 10 30k constellations and run in parallel (or whatever) and then just concatenate the indx arrays.

In [8]:
print(indx.size)
indx

36


array([4290, 4354, 2140, 2871, 2761, 1216, 4068, 3046, 4909,  480, 4189,
       5754, 5158, 3860, 1261, 3000, 3085, 1385, 2466, 3368, 3338, 1890,
       2897, 1719, 1087, 1712,  447, 1914,  439, 3782, 2467, 4356, 5678,
       3834, 4060,   55])

In [9]:
# so, we really didn't lose that many observations at all, 27 out of 6810
# let's drop those out of the dataframe
final_data = ss_objects.drop(index=indx)

In [10]:
final_data.shape

(6067, 27)

In [11]:
# And now we can put that back as a new txt file with the streaked objects removed
final_data.to_csv('baseline_v3.0_10yrs__vatiras_granvik_10k_obs_streaked.txt', index=False, sep=' ')

In [12]:
36/6783

0.005307386112339673

In [ ]:
#Wow, I think this exactly matches what we had in the paper as well. 0.4% loss of detections. 